**Loading** CMU-MOSI **dataset**

In [1]:
# !git clone https://github.com/pliang279/MultiBench.git

Cloning into 'MultiBench'...
remote: Enumerating objects: 6943, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6943 (delta 72), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6943/6943), 51.07 MiB | 2.93 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.


In [1]:
# %cd MultiBench

/home/hi-born4/6th Sem/Applied Linear Algebra for Machine Learning/Tensor fusion network for multi modal sentiment analysis/MultiBench


In [2]:
# !mkdir data
# !pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU

In [2]:
import torch
import sys
import os

In [6]:
# Import the associated dataloader for affect datasets, which MOSI is a part of.
from datasets.affect.get_data import get_dataloader

# Create the training, validation, and test-set dataloaders.
traindata, validdata, testdata = get_dataloader(
    'mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

ImportError: dynamic module does not define module export function (PyInit__torchtext)

In [8]:
for batch in traindata:
  print(batch)

NameError: name 'traindata' is not defined

----------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F

class SubnetModel(nn.Module):
    def __init__(self,input_size,num_utterances,fc1_size, fc2_size,fc3_size,dropout=0.15):
        super(SubnetModel, self).__init__()

        # self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        # self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)
        # # self.weight_decay=weight_decay

        # self.norm = nn.BatchNorm1d(input_size)
        self.drop = nn.Dropout(p=dropout)

        # Fully connected layers

        #fc1 gets hidden_size dimension values as input
        self.fc1 = nn.Linear(input_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, fc3_size)

        # Output layer
        # self.output_layer = nn.Linear(fc3_size, 1)

        # Activation functions
        self.relu = nn.ReLU()
        # self.sigmoid=nn.Sigmoid()


    def forward(self, x):

        # x=x.view(x.size(0),-1) # flattening the input to feed to the fully connected layer as it expects 1D input
        # print(x.shape)
        # normed = self.norm(x)
        # dropped = self.drop(normed)

        # x = F.normalize(x, p=2, dim=1)
        x = torch.mean(x, dim=1)

        # print(x.shape)

        nan_mask = torch.isnan(x)
        # Replace NaN values with zeros
        x[nan_mask] = 0

        # normed = self.norm(x)

        fc1_out = self.relu(self.fc1(x))

        fc2_out = self.relu(self.fc2(fc1_out))
        drop=self.drop(fc2_out)

        fc3_out=self.relu(self.fc3(drop))

        return fc3_out

        # # Output layer with Sigmoid activation
        # output = self.sigmoid(self.output_layer(fc3_out))

        # # get output between -3 and +3

        # output=output*self.output_range+self.output_shift


        # return output

    # def l2_regularization_loss(self):
    #     l2_loss = 0.0
    #     for param in self.parameters():
    #         l2_loss += torch.norm(param, p=2)  # L2 norm of the parameters
    #     return self.weight_decay * l2_loss

In [ ]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

class TextModel(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers,fc1_size, fc2_size):
        super(TextModel, self).__init__()

        # self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        # self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

        # LSTM layer (stacked LSTM)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)

        # Fully connected layers

        #fc1 gets hidden_size dimension values as input
        self.fc1 = nn.Linear(hidden_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)

        self.dropout=nn.Dropout(p=0.15)


        # Output layer
        # self.output_layer = nn.Linear(fc2_size, 1)

        # Activation functions
        self.relu = nn.ReLU()
        # self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        # x = x.view(-1, x.size(-1), x.size(-2))

        # LSTM layer

        lstm_out, (hidden_states, cell_states) = self.lstm(x)
        # last_time_step_index = lstm_out.size(1) - 1

        # # # Extract the output of the final LSTM unit
        # final_lstm_output = lstm_out[:, last_time_step_index, :]

        # print(lstm_out.shape)
        # print(hidden_states.shape)
        # print(type(hidden_states))

        #print(hidden_states.squeeze().shape)

        fc1_out = self.relu(self.fc1(hidden_states.squeeze()))
        drop=self.dropout(fc1_out)

        fc2_out = self.relu(self.fc2(drop))

        return fc2_out

        # # Output layer with Sigmoid activation
        # output = self.sigmoid(self.output_layer(fc2_out))
        # #get output between -3 and +3
        # output=output*self.output_range+self.output_shift

        # return output

In [ ]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.autograd import Variable

class TFN(nn.Module):
    def __init__(self,audio_params,video_params,text_params,SIN_params):
        super(TFN, self).__init__()

        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

        self.audio_params=audio_params
        self.video_params=video_params
        self.text_params=text_params

        #unimodels
        self.audio_subnet=SubnetModel(audio_params[0],audio_params[1],audio_params[2],audio_params[3],audio_params[4])
        self.video_subnet=SubnetModel(video_params[0],video_params[1],video_params[2],video_params[3],video_params[4])
        self.text_subnet=TextModel(text_params[0],text_params[1],text_params[2],text_params[3],text_params[4])

        # Fully connected layers

        #fc1 gets hidden_size dimension values as input
        self.fc1 = nn.Linear(((audio_params[2]+1)*(video_params[2]+1)*(text_params[3]+1)), SIN_params[0])
        self.fc2 = nn.Linear(SIN_params[0], SIN_params[1])

        # Output layer
        self.output_layer = nn.Linear(SIN_params[1], 1)

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):

        DTYPE = torch.FloatTensor

        batch_size=x[0].shape[0]

        # unimodal outputs

        audio_out=self.audio_subnet(x[0])
        video_out=self.video_subnet(x[1])
        text_out=self.text_subnet(x[2])

        # adding 1 to increase the dimension value

        audio_out = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), audio_out), dim=1)
        # print("audio_out")
        # print(audio_out.shape)
        video_out = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), video_out), dim=1)
        # print("video_out")
        # print(video_out.shape)
        text_out = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), text_out), dim=1)
        # print("text_out")
        # print(text_out.shape)


        # tensorfusion operation

        fusion_tensor = torch.bmm(audio_out.unsqueeze(2), video_out.unsqueeze(1))
        # print("fusion tensor 1")
        # print(fusion_tensor.shape)
        fusion_tensor = fusion_tensor.view(-1, (self.audio_params[2] + 1) * (self.video_params[2] + 1), 1)
        # print("fusion tensor 2")
        # print(fusion_tensor.shape)
        fusion_tensor = torch.bmm(fusion_tensor, text_out.unsqueeze(1)).view(batch_size, -1)
        # print("fusion tensor 3")
        # print(fusion_tensor.shape)


        # sentiment inference network (SIN)

        fc1_out = self.relu(self.fc1(fusion_tensor))

        fc2_out = self.relu(self.fc2(fc1_out))

        # Output layer with Sigmoid activation
        output = self.sigmoid(self.output_layer(fc2_out))

        # print("output")
        # print("output.shape")

        # get output between -3 and +3
        output=output*self.output_range+self.output_shift

        return output

**-------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
audio_params=(35,50,32,32,32) # (feature_length,max_seq_len,fc1_size,fc2_size,fc3_size)
video_params=(74,50,32,32,32) # (feature_length,max_seq_len,fc1_size,fc2_size,fc3_size)
text_params=(300,128,1,128,128) # (feature_length,LSTM_hidden_size,num_LSTM_layers,fc1_size,fc2_size)

SIN_params=(128,128)

final_model=TFN(audio_params,video_params,text_params,SIN_params)

In [ ]:
Loss = torch.nn.MSELoss() # mean absolute error
optimizer = torch.optim.Adam(list(final_model.parameters())[2:],lr=0.0005,weight_decay=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
num_epochs = 100

In [ ]:
from sklearn.metrics import r2_score
import numpy as np

In [ ]:
for epoch in range(num_epochs):

    print("EPOCH : ",epoch+1)
    # Training
    total_train_loss=0.0
    num_sequences=0
    final_model.train()  # Set the model to training mode
    for batch in traindata:
        # targets=targets.unsqueeze(1).repeat(1, 50, 1)
        optimizer.zero_grad()  # Zero the gradients
        outputs = final_model(batch[:-1])
        loss = Loss(outputs, batch[-1])
        total_train_loss+=loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        num_sequences+=1

    average_train_loss = total_train_loss / num_sequences
    print("-------------Training----------------")
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_train_loss:.4f}')

    # Validation
    final_model.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    num_sequences=0
    val_all_predictions = []
    val_all_targets = []
    with torch.no_grad():
        best_val_loss = np.inf
        patience=3
        current_patience = patience

        for batch in validdata:
            val_targets=batch[-1]
            val_outputs = final_model(batch[:-1])
            val_loss = Loss(val_outputs, val_targets)
            total_val_loss += val_loss.item()

            # Calculate R2 score
            val_targets_np = val_targets.cpu().numpy()
            val_outputs_np = val_outputs.cpu().numpy()


            val_all_predictions.extend(val_outputs_np)
            val_all_targets.extend(val_targets_np)
            num_sequences+=1

    average_val_loss = total_val_loss / num_sequences
    r_squared = r2_score(val_all_targets, val_all_predictions)

    print("--------------Validation----------")
    print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}, R2 Score: {r_squared:.4f}')

    # if average_val_loss < best_val_loss:
    #     best_val_loss=average_val_loss
    #     current_patience=patience
    # else:
    #     current_patience-=1
    #     if current_patience==0:
    #       print("Model performance degarding , Early stopping!!")
    #       break



    print("\n \n")


EPOCH :  1
-------------Training----------------
Epoch [1/100], Loss: 2.3177
--------------Validation----------
Epoch [1/100], Validation Loss: 2.6593, R2 Score: -0.0340

 

EPOCH :  2
-------------Training----------------
Epoch [2/100], Loss: 2.2154
--------------Validation----------
Epoch [2/100], Validation Loss: 2.6288, R2 Score: -0.0189

 

EPOCH :  3
-------------Training----------------
Epoch [3/100], Loss: 2.2167
--------------Validation----------
Epoch [3/100], Validation Loss: 2.5608, R2 Score: 0.0122

 

EPOCH :  4
-------------Training----------------
Epoch [4/100], Loss: 2.1827
--------------Validation----------
Epoch [4/100], Validation Loss: 2.6125, R2 Score: -0.0118

 

EPOCH :  5
-------------Training----------------
Epoch [5/100], Loss: 2.1128
--------------Validation----------
Epoch [5/100], Validation Loss: 2.6083, R2 Score: -0.0055

 

EPOCH :  6
-------------Training----------------
Epoch [6/100], Loss: 2.0593
--------------Validation----------
Epoch [6/100], Vali

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

final_model.eval()

with torch.no_grad():
    all_predictions = []
    all_targets = []

    for batch in testdata:
        test_targets=batch[-1]
        test_outputs = final_model(batch[:-1])

        # Convert predictions and targets to numpy arrays
        predictions = test_outputs.numpy().flatten()
        targets = test_targets.numpy().flatten()

        all_predictions.extend(predictions)
        all_targets.extend(targets)


    mae = mean_absolute_error(all_targets, all_predictions)
    r_squared = r2_score(all_targets, all_predictions)

    print(f'Test MAE: {mae:.4f}, R-squared: {r_squared:.4f}')


Test MAE: 1.4757, R-squared: -0.2117


----------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------

# Saving the model:


In [ ]:
torch.save(final_model.state_dict(), '/content/drive/MyDrive/multi_model_SA/TFN_MAE-1.2386_R-0.0171.pth')

------------------------------------------------------------